# GoDaddy - Microbusiness Density Forecasting


In [1]:
import os
import sys
import pandas as pd
import numpy as np

from word2number import w2n

pd.set_option('display.max_columns', 500)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score ###???###
from sklearn.metrics import mean_absolute_error, make_scorer ###???###

#Standard plotly imports
import plotly
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objs as go

import plotly.graph_objects as go
from plotly.offline import iplot

import seaborn as sns
import matplotlib.pyplot as plt

# Using plotly + cufflinks in offline mode
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf

cf.set_config_file(offline=True)
init_notebook_mode(connected=True)

## To display multiple output from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [ ]:
train["first_day_of_month"] = pd.to_datetime(train["first_day_of_month"])
train = train.sort_values(['cfips','first_day_of_month']).reset_index(drop=True)
test["first_day_of_month"] = pd.to_datetime(test["first_day_of_month"])
test = test.sort_values(['cfips','first_day_of_month']).reset_index(drop=True)

In [ ]:
train.info()
test.info()

In [ ]:
train.describe()
test.describe()

In [ ]:
train.isnull().sum()

#### The train data does not have any null values. 
#### A bird eye view of the train-data is shown in the form of microdensity scatter plot w.r.t date-time

In [ ]:
fig = px.scatter(train, x='first_day_of_month',y='microbusiness_density', color="county", symbol="county")
fig.show()

In [ ]:
### The plot shows several data jumps in several of the counties. and most predominantly 
### between data points at date 01Jan2021 and 01Feb2021.
### Use bayesoffline to find the points of data shifts.

In [ ]:
import sdt.changepoint as c
f=open("changepoint2.txt", "w")
BayesOffline = c.BayesOffline()

for cfips in train["cfips"].unique():
        df_cfips = train[train["cfips"] == cfips]
    
        values = df_cfips["microbusiness_density"].values

        # Out is a list of possible changepoint "indices"
        out = BayesOffline.find_changepoints(values, 0.9);

        # If there is a changepoint.
        if len(out) > 0:
            f.write(str(cfips) + ' ')
            f.write(str(out) + '\n')
f.close();

In [ ]:
### To select only data shifts corresponding to the dates 01-Jan-21 and 01-Feb-2021
f=open("changepoint1.txt","r")
cfips17=list()
for line in f:    
    words=line.split()
    #clusters = {}
    if "17" or "18" in words[1] :
        cfips17.append(w2n.word_to_num(words[0]))   
f.close()

In [ ]:
train17=train[train['cfips'].isin(cfips17)]

In [ ]:
fig = px.scatter(train17, x='first_day_of_month',y='microbusiness_density', color="county", symbol="county")
fig.show()

In [ ]:
### To adjust for the shifts due to methodology change at 01-Jan-2021
def shiftadjust(timeseries):
    """
    Removes the level shift in the timeseries data between
    the 17th (2021-01-01) and the 18th (2021-02-01) indices.    
    """
    series = timeseries.values.copy() 
    adjust_by = series[18] - series[17] 
    series[:18] += adjust_by    
    return series
train17["microbusiness_density"] = train17.groupby("cfips")["microbusiness_density"].transform(shiftadjust)

In [ ]:
train17

In [ ]:
train.loc[train.cfips.isin(train17.cfips), ['microbusiness_density']] = train17[['microbusiness_density']]

train.to_csv('trainShiftcorrected.csv')

In [ ]:
trainSC=pd.read_csv('trainShiftcorrected.csv')


In [ ]:
fig = px.scatter(trainSC, x='first_day_of_month',y='microbusiness_density', color="county", symbol="county")
fig.show()

In [ ]:
InteractiveShell.ast_node_interactivity = "last_expr_or_assign"

In [ ]:
# Adding a parameter Linearity to easy recognition of cfips with linear trend.
# Also not using the population cut-off parameter to define threshold. 

DISPLAY = 0
THRESHOLD = 0.103 #this is 10.3% same value as 8/78 from old notebook
ACTIVE_THRESHOLD = 0_000

IDS = trainSC.cfips.unique()
x_trainSC = np.arange(39).reshape((-1,1))
x_test = np.arange(38,47).reshape((-1,1))

preds = np.zeros((len(IDS),8))
linearity = np.zeros((len(IDS),8))
last_preds = np.zeros((len(IDS),8))
lin_trend = 0

ct = 0
for i,c in enumerate(IDS):
    df = trainSC.loc[trainSC.cfips==c]
    last = df.microbusiness_density.values[-1]
    active = df.active.values[-1]
    last_preds[i,] = [last]*8
    
    # FIT LINEAR REGRESSION
    model = LinearRegression()
    model.fit(x_trainSC,df.microbusiness_density)
    p = model.predict(x_trainSC)
    
    # COMPUTE trainSC ERROR
    err = p - df.microbusiness_density.values
    rng = df.microbusiness_density.max() - df.microbusiness_density.min()
    
    # DETERMIN IF TIME SERIES IS LINEAR OR NOT
    s = 0
    for k in range(39):
        e = np.abs( err[k] )
        r = e/rng # absolute error divided by range
        s += r
    s = s/39 # now S is MAPE mean absolute percentage error
    
    # IF S <= 10% THEN WE ASSUME THIS COUNTY HAS A LINEAR TREND
    if (s>THRESHOLD)|(active<ACTIVE_THRESHOLD): 
        preds[i,] = [last]*8
        linearity[i,] = 0.0
        continue
        
    # INFER TEST DATA WITH LINEAR REGRESSION
    p2 = model.predict(x_test)
    shift =  last - p2[0]
    preds[i,] = p2[1:]+shift
    linearity[i,] = 1.0
    
    # COUNT STUFF
    ct += 1
    lin_trend += 1
    if ct>=DISPLAY+1: continue
        
    # PLOT STUFF
    plt.figure(figsize=(20,5))
    plt.plot(df.first_day_of_month,df.microbusiness_density,'-o',label='trainSC data')
    plt.plot(df.first_day_of_month,p,'--',label='linear regression')
    plt.plot(test.first_day_of_month.values[:8],preds[i,],'-o',label='test pred')
    pre='YES, we WILL USE linear regression for\n'
    plt.title(f'{pre}CFIPS {c}',size=18)
    plt.xlabel('Date',size=16)
    plt.ylabel('Microbusiness Density',size=16)
    plt.legend()
    plt.show()
    
    plt.hist(err,bins=20,label='error')
    plt.plot([-rng/2,-rng/2],[0,10],'--',color='black',label='range')
    plt.plot([rng/2,rng/2],[0,10],'--',color='black')
    plt.xlim((-rng * 0.75,rng * 0.75))
    plt.legend()
    plt.title(f'trainSC Error vs. trainSC Range. (avg={100*s:2.1f}%)',size=18)
    plt.show()
    print('\n\n\n\n\n\n')
    
# PRINT HOW MANY LINEAR TIME SERIES WE FOUND
print(f'There are {lin_trend} counties with a linear trend.')

In [ ]:
test['microbusiness_density'] = preds.reshape((-1))
test['Linearity'] = linearity.reshape((-1))

In [ ]:
test.shape
trainSC.shape

In [ ]:
#### Some EDA of the data after Linear Regression model fitting using only 
#### Shift corrected and the data with linear trends only.

In [ ]:
## To plot the counties with and without linear trends 
train2=trainSC[trainSC['cfips'].isin(test.loc[test.Linearity == 1.0, 'cfips'])]
train3=trainSC[~trainSC['cfips'].isin(test.loc[test.Linearity == 1.0, 'cfips'])]
test2=test[test['cfips'].isin(test.loc[test.Linearity == 1.0, 'cfips'])]
test3=test[~test['cfips'].isin(test.loc[test.Linearity == 1.0, 'cfips'])]

In [ ]:
## Counties that were fit with Linear Regression.
fig = px.scatter(train2, x='first_day_of_month',y='microbusiness_density', color="county", symbol="county")
fig.show()

In [ ]:
## Counties that were NOT fit with Linear Regression
fig = px.scatter(train3, x='first_day_of_month',y='microbusiness_density', color="county", symbol="county")
fig.show()

In [ ]:
# plot the train and predicted (test) data whose linearity is 1.0
fig = go.Figure()
dfs = {"df1" : train2, "df2": test2}
for i in dfs:
    fig = fig.add_trace(go.Scatter(x = dfs[i]["first_day_of_month"],
                                   y = dfs[i]["microbusiness_density"], 
                                   mode = 'markers',
                                   name = i))
fig.show()

In [ ]:
# plot the train and predicted (test) data whose linearity is 1.0
fig = go.Figure()
dfs = {"df1" : train3, "df2": test3}
for i in dfs:
    fig = fig.add_trace(go.Scatter(x = dfs[i]["first_day_of_month"],
                                   y = dfs[i]["microbusiness_density"], 
                                   mode = 'markers',
                                   name = i))
fig.show()

In [ ]:
sub = test[['row_id','microbusiness_density']]
sub.to_csv('submission.csv',index=False)

#### Some Exploratory Data Analysis

In [ ]:
fig = plt.figure(figsize=(12,12))
sns.boxplot(data=train2, y="state", x="microbusiness_density",color="teal")
plt.ylabel("")
plt.show()

In [ ]:
fig = plt.figure(figsize=(12,12))
sns.boxplot(data=train3, y="state", x="microbusiness_density",color="red")
plt.ylabel("")
plt.show()

In [ ]:
sort_std = train2.groupby(['state']).describe()['microbusiness_density'].sort_values('std').index
each_state = train2.groupby(['state']).describe()['microbusiness_density'].sort_values('std')
each_state

In [ ]:
#len(list(set(train.cfips) & set(train5.cfips)))

In [ ]:
fig = px.scatter(train3, x='first_day_of_month',y='microbusiness_density', color="county", symbol="county")
fig.show()


#### transition from August 2019 to October 2022 with 3 state that has most std 

In [ ]:
fig = px.scatter(train2[train2['state'].isin(['Wyoming','Colorado','Iowa'])], x='first_day_of_month',y='microbusiness_density', color="county", symbol="county")
fig.show()


#### transition from August 2019 to October 2022 with 3 state that has least std 

In [ ]:
fig = px.scatter(train2[train2['state'].isin(['Massachusetts','New Hampshire','Nevada'])], x='first_day_of_month',y='microbusiness_density', color="county", symbol="county")
fig.show()
